# Clasificador de supervivencia de pasajeros del Titanic

Este notebook construye un **pipeline de scikit-learn** (imputación, codificación One-Hot, escalado y Reg.Logística) para predecir la **probabilidad de supervivencia** en el dataset clásico del Titanic.  
Se evalúa con métricas de clasificación (F1, accuracy, precision, recall, ROC-AUC) y se selecciona un **umbral óptimo por F1**. Finalmente se exporta un **artefacto (.pkl)** con el modelo y metadatos para usarlo en la API.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, f1_score, precision_score, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

## Obtención del dataset (OpenML)

Usamos `fetch_openml` de **scikit-learn** para descargar el conjunto **titanic** desde OpenML:

- `as_frame=True` → entrega los datos como `pandas.DataFrame`/`Series`.
- `return_X_y=True` → retorna directamente `X_titanic` (características) y `y_titanic` (objetivo).
- El objetivo es **`survived`** (0 = no sobrevive, 1 = sobrevive).

El `DataFrame` incluye muchas columnas crudas (p. ej. `name`, `ticket`, `cabin`, `home.dest`, etc.).  
En este trabajo **solo utilizaremos** las siguientes variables predictoras:

- `pclass`, `sex`, `age`, `sibsp`, `parch`, `fare`, `embarked`.

El resto se considera **ruido** o **potencial fuga de información** (leakage) y se descarta.  
Abajo imprimimos las columnas y un `head()` para una inspección rápida antes del preprocesamiento.


In [ ]:
X_titanic, y_titanic = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
print(X_titanic.columns)

Index(['pclass', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare',
       'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


In [ ]:
X_titanic.head(5)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


### 1) Imports
Cargamos librerías de ciencia de datos (**numpy, pandas, joblib**) y de **scikit-learn** para armar el pipeline, entrenar el modelo y calcular métricas (F1, accuracy, precision, recall, ROC-AUC).

---

### 2) Configuración de columnas
Definimos las variables que usará el modelo:
- **features**: `['pclass','sex','age','sibsp','parch','fare','embarked']`
- Separamos numéricas y categóricas para el preprocesamiento.

---

### 3) Preprocesamiento + Pipeline
- Numéricas: imputación (mediana) + estandarización.
- Categóricas: imputación (moda) + One-Hot Encoding.
- Clasificador: **LogisticRegression** dentro de un **Pipeline** completo.

---

### 4) Train/Test split
Partimos los datos 80/20 con `train_test_split` (semilla 42) y estratificación en el objetivo para mantener proporciones.

---

### 5) Entrenamiento
Entrenamos el pipeline con `clf.fit(X_train, y_train)`.

---

### 6) Métricas (umbral 0.5)
Calculamos predicciones y probabilidades con el umbral por defecto (0.5) y reportamos F1, accuracy, precision, recall, ROC-AUC y matriz de confusión.

---

### 7) Búsqueda de umbral (max F1)
Barrido simple de umbrales entre 0.1 y 0.9; escogemos el que **maximiza F1** en el set de test y mostramos las métricas resultantes.

---

### 8) Guardar artefacto
Guardamos **un solo archivo** `model/logistic_titanic_pipeline.pkl` con:
- `model`: el Pipeline entrenado  
- `threshold`: umbral óptimo  
- `features`: columnas esperadas  
Incluye un sanity check de carga para verificar las llaves.


In [ ]:
# =========================
# 1) Imports
# =========================
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    f1_score, accuracy_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_auc_score
)

# =========================
# 2) Configuración de columnas
# =========================
# Ajusta estos nombres a los de tu DataFrame X_titanic
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']

numeric_features = ['age', 'sibsp', 'parch', 'fare']
categorical_features = ['pclass', 'sex', 'embarked']

# =========================
# 3) Preprocesadores y Pipeline
# =========================
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),   # rellena nulos con mediana
    ('scaler', StandardScaler())                     # escala numéricas
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # rellena nulos con moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # one-hot encoding
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# =========================
# 4) Train / Test split
# =========================
# y_titanic debería ser binaria (0/1). Si no lo es, intenta castear:
try:
    y_titanic = y_titanic.astype(int)
except Exception:
    pass

X_train, X_test, y_train, y_test = train_test_split(
    X_titanic[features], y_titanic, test_size=0.2, random_state=42, stratify=y_titanic
)

# =========================
# 5) Entrenamiento
# =========================
clf.fit(X_train, y_train)

# =========================
# 6) Métricas con umbral por defecto (0.5)
# =========================
y_pred_05 = clf.predict(X_test)                                   # usa 0.5 internamente
y_proba   = clf.predict_proba(X_test)[:, 1]                       # prob de clase positiva (sobrevive=1)

print("=== Métricas (umbral 0.5) ===")
print("F1:", f1_score(y_test, y_pred_05))
print("Accuracy:", accuracy_score(y_test, y_pred_05))
print("Precision:", precision_score(y_test, y_pred_05))
print("Recall:", recall_score(y_test, y_pred_05))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_05))
print(classification_report(y_test, y_pred_05))

# =========================
# 7) (Opcional) Buscar umbral que maximiza F1 en X_test
#    *Para un trabajo más “purista”, separa un set de validación.
# =========================
thresholds = np.linspace(0.1, 0.9, 33)
f1s = []
for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    f1s.append((t, f1_score(y_test, y_pred_t)))

best_threshold, best_f1 = max(f1s, key=lambda x: x[1])

print("\n=== Búsqueda de umbral por F1 (en test) ===")
print(f"Mejor umbral: {best_threshold:.3f}  |  F1: {best_f1:.4f}")

# Métricas con el mejor umbral encontrado
y_pred_best = (y_proba >= best_threshold).astype(int)
print("\n=== Métricas con umbral óptimo (F1) ===")
print("F1:", f1_score(y_test, y_pred_best))
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print("Precision:", precision_score(y_test, y_pred_best))
print("Recall:", recall_score(y_test, y_pred_best))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred_best))

# =========================
# 8) Guardar artefacto (modelo + umbral + features) en UN SOLO archivo
# =========================
os.makedirs("model", exist_ok=True)

artifact = {
    "model": clf,                   # el Pipeline entrenado
    "threshold": float(best_threshold),
    "features": features           # ['pclass','sex','age','sibsp','parch','fare','embarked']
}

joblib.dump(artifact, "model/logistic_titanic_pipeline.pkl")
print("Artefacto guardado en: model/logistic_titanic_pipeline.pkl")

# (Sanity check rápido)
_loaded = joblib.load("model/logistic_titanic_pipeline.pkl")
print("Llaves:", list(_loaded.keys()))
# Deberías ver: ['model', 'threshold', 'features']

=== Métricas (umbral 0.5) ===
F1: 0.7357512953367875
Accuracy: 0.8053435114503816
Precision: 0.7634408602150538
Recall: 0.71
ROC-AUC: 0.8673148148148149
Matriz de confusión:
 [[140  22]
 [ 29  71]]
              precision    recall  f1-score   support

           0       0.83      0.86      0.85       162
           1       0.76      0.71      0.74       100

    accuracy                           0.81       262
   macro avg       0.80      0.79      0.79       262
weighted avg       0.80      0.81      0.80       262


=== Búsqueda de umbral por F1 (en test) ===
Mejor umbral: 0.550  |  F1: 0.7634

=== Métricas con umbral óptimo (F1) ===
F1: 0.7634408602150538
Accuracy: 0.8320610687022901
Precision: 0.8255813953488372
Recall: 0.71
Matriz de confusión:
 [[147  15]
 [ 29  71]]
Artefacto guardado en: model/logistic_titanic_pipeline.pkl
Llaves: ['model', 'threshold', 'features']


### Cargar artefacto + predicción de ejemplo

**Objetivo:** usar el artefacto persistido para hacer una predicción puntual.

1. **Carga del artefacto**  
   - Se intenta cargar `model/logistic_titanic_pipeline.pkl` como **diccionario unificado** con:
     - `model` → Pipeline entrenado (preprocesa + clasifica)
     - `threshold` → umbral óptimo (F1)
     - `features` → columnas esperadas (ordenadas)
   - **Fallback legado:** si el `.pkl` no trae diccionario, se carga el modelo y, si existe, `model/logistic_titanic_meta.pkl` para obtener `threshold` y `features`.

2. **Payload de ejemplo**  
   - Se crea un `DataFrame` con los campos crudos (`pclass, sex, age, sibsp, parch, fare, embarked`).
   - El **Pipeline** se encarga de imputar, escalar y hacer One-Hot Encoding.

3. **Predicción**  
   - Se calcula la **probabilidad** de clase positiva con `predict_proba` (sobrevive = 1).
   - Se aplica el **umbral** (`threshold`) para obtener la etiqueta final (`0/1`).
   - Se imprime: probabilidad, decisión (`Sobrevive / No sobrevive`) y el umbral usado.


In [ ]:
import os
import joblib
import pandas as pd

ARTIFACT_PATH = "model/logistic_titanic_pipeline.pkl"
META_PATH     = "model/logistic_titanic_meta.pkl"  # fallback legacy

# 1) Cargar artefacto unificado (modelo + threshold + features)
art = joblib.load(ARTIFACT_PATH)

if isinstance(art, dict) and "model" in art:
    pipe      = art["model"]
    thr       = float(art.get("threshold", 0.5))
    FEATURES  = art.get("features", ['pclass','sex','age','sibsp','parch','fare','embarked'])
else:
    # Fallback (por si todavía tienes archivos viejos separados)
    pipe = art
    try:
        meta     = joblib.load(META_PATH)
        thr      = float(meta.get("threshold", 0.5))
        FEATURES = meta.get("features", ['pclass','sex','age','sibsp','parch','fare','embarked'])
    except Exception:
        thr      = 0.5
        FEATURES = ['pclass','sex','age','sibsp','parch','fare','embarked']

# 2) Pasajero de ejemplo (el Pipeline hace el preprocesamiento)
ejemplo = pd.DataFrame([{
    "pclass": 1,
    "sex": "female",
    "age": 20,
    "sibsp": 0,
    "parch": 1,
    "fare": 80.0,
    "embarked": "C"
}])

# 3) Predecir probabilidad y aplicar umbral
p = float(pipe.predict_proba(ejemplo[FEATURES])[:, 1][0])
y_hat = int(p >= thr)

print(f"Ejemplo → Probabilidad de sobrevivir: {p:.3f}  → "
      f"{'Sobrevive' if y_hat==1 else 'No sobrevive'} (umbral={thr:.2f})")


Ejemplo → Probabilidad de sobrevivir: 0.963  → Sobrevive (umbral=0.55)
